# CH-03 股市資料蒐集、爬蟲與搭建資料庫

## 3-2 資料爬蟲

## 證交所資料爬蟲
1. 進入證交所網址：https://www.twse.com.tw/zh/index.html
2. 使用開發者模式取得請求資料網址

### 1️⃣ 匯入套件

In [1]:
import requests
import pandas as pd
import datetime as dt # 時間套件
from dateutil.relativedelta import relativedelta

### 2️⃣ 取得個股日成交資訊

In [2]:
# 輸入股票代號
stock_id = '2330'
# 當日時間
date = dt.date.today().strftime("%Y%m%d")
# 取得證交所網站資料
stock_data = requests.get(f'https://www.twse.com.tw/rwd/zh/ \
            afterTrading/STOCK_DAY?date={date}&stockNo={stock_id}')
json_data = stock_data.json()
df = pd.DataFrame(data=json_data['data'],
                  columns=json_data['fields'])
df.tail()

,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數
8,113/05/14,"29,663,617","24,309,306,615",816.00,825.00,811.00,825.00,+6.00,"34,685"
9,113/05/15,"41,805,778","35,112,739,055",838.00,844.00,837.00,839.00,+14.00,"57,892"
10,113/05/16,"46,276,890","39,179,826,883",852.00,856.00,837.00,841.00,+2.00,"60,617"
11,113/05/17,"27,855,719","23,373,478,247",848.00,848.00,834.00,835.00,-6.00,"38,718"
12,113/05/20,"30,148,330","25,069,342,221",834.00,838.00,822.00,835.00,0.00,"43,094"


### 3️⃣ 取得連續月份資料
以個股本益比為例

In [3]:
# 設定抓取幾個月資料
month_num=3
date_now = dt.datetime.now()

# 建立日期串列
date_list = [(date_now - relativedelta(months=i)).replace(day=1).\
             strftime('%Y%m%d') for i in range(month_num)]

date_list.reverse()
all_df = pd.DataFrame()

# 使用迴圈抓取連續月份資料
for date in date_list:
  url = f'https://www.twse.com.tw/rwd/zh/afterTrading/\
      STOCK_DAY?date={date}&stockNo={stock_id}'
  try:
    json_data = requests.get(url).json()
    df = pd.DataFrame(data=json_data['data'],
                  columns=json_data['fields'])
    all_df = pd.concat([all_df, df], ignore_index=True)
  except Exception as e:
    print(f"無法取得{date}的資料, 可能資料量不足.")

all_df.head()

,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數
0,113/03/01,"24,167,721","16,699,995,060",697.00,697.00,688.00,689.00,-1.00,"26,282"
1,113/03/04,"97,210,112","69,868,348,694",714.00,725.00,711.00,725.00,+36.00,"125,799"
2,113/03/05,"73,299,411","53,751,887,376",735.00,738.00,728.00,730.00,+5.00,"69,851"
3,113/03/06,"52,464,833","38,203,868,985",718.00,738.00,717.00,735.00,+5.00,"49,897"
4,113/03/07,"80,382,406","61,221,034,146",755.00,769.00,754.00,760.00,+25.00,"96,348"


## 用 BeautifulSoup4 取得 Yahoo 股市資料

###4️⃣ 匯入套件

In [4]:
from datetime import datetime
from bs4 import BeautifulSoup
import time

###  5️⃣ 取得當日股價

In [5]:
def yahoo_stock(stock_id):
    url = f'https://tw.stock.yahoo.com/quote/{stock_id}.TW'
    # 使用 requests 取得網頁內容
    response = requests.get(url)
    html = response.content
    # 使用 Beautiful Soup 解析 HTML 內容
    soup = BeautifulSoup(html, 'html.parser')
    # 使用 find 與 find_all 定位元素
    time_element = soup.find('section',\
                {'id': 'qsp-overview-realtime-info'}).find('time')
    table_soups = soup.find('section',\
                {'id': 'qsp-overview-realtime-info'}).find('ul')\
                                   .find_all('li')
    fields = []
    datas = []
    for table_soup in table_soups:
        table_datas = table_soup.find_all('span')
        for num,table_data in enumerate(table_datas):
            if table_data.text =='':
                continue
            if num == 0:
                fields.append(table_data.text)
            else:
                datas.append(table_data.text)
    # 建立 DataFrame
    df = pd.DataFrame([datas], columns=fields)
    # 增加日期和股號欄位
    df.insert(0,'日期',time_element['datatime'])
    df.insert(1,'股號',stock_id)
    # 回傳 DataFrame
    return df

yahoo_stock(stock_id)

,日期,股號,成交,開盤,最高,最低,均價,成交金額(億),昨收,漲跌幅,漲跌,總量,昨量,振幅
0,2024/05/21 10:01,2330,834,830,840,830,834,48.42,835,0.12%,1.00,"5,806","28,777",1.20%


### 6️⃣ 取得季報表資訊


In [6]:
# 函式可用於奇摩財報
def url_find(url):
    words = url.split('/')
    k = words[-1]
    # 使用requests取得網頁內容
    response = requests.get(url)
    html = response.content
    # 使用Beautiful Soup解析HTML內容
    soup = BeautifulSoup(html, 'html.parser')
    # 找到表格的表頭
    table_soup = soup.find('section', {'id': 'qsp-{}-table'.format(k)})
    table_fields=table_soup.find('div', class_='table-header')
    table_fields_lines = list(table_fields.stripped_strings)
    # 找到數據
    data_rows = table_soup.find_all('li' ,class_='List(n)')
    # 解析資料行內容
    data = []
    for row in data_rows:
        row_data = list(row.stripped_strings)
        data.append(row_data)
    # 建立 DataFrame
    df = pd.DataFrame(data, columns=table_fields_lines)
    return df

# 抓損益表
url = f'https://tw.stock.yahoo.com/quote/{stock_id}/income-statement'
# 抓資產負債表
# url = f'https://tw.stock.yahoo.com/quote/{stock_id}/balance-sheet'
# 抓現金流量表
# url = f'https://tw.stock.yahoo.com/quote/{stock_id}/cash-flow-statement'

# 抓取季報表資料
df = url_find(url).transpose()

# 資料處理
df.columns = df.iloc[0]
df = df[1:]
df.insert(0,'年度/季別',df.index)
df.columns.name = None
df.reset_index(drop=True, inplace=True)

df.tail()

,年度/季別,營業收入,營業毛利,營業費用,營業利益,稅後淨利
15,2020 Q2,"310,698,367","164,692,825","33,520,709","131,094,538","120,892,514"
16,2020 Q1,"310,597,183","160,784,181","32,323,046","128,521,637","117,062,893"
17,2019 Q4,"317,237,065","159,240,985","34,942,621","124,243,722","116,078,194"
18,2019 Q3,"293,045,439","139,432,161","31,378,953","107,887,292","101,102,454"
19,2019 Q2,"240,998,475","103,673,230","27,164,995","76,304,053","66,775,851"


##使用 selenium 做新聞爬蟲

### 7️⃣ 用 requests 取得股票新聞

In [7]:
# 股票代號
stock_id = "2330"
# 預設表格數據和欄位
field=['股號','日期','標題','內容']
data=[]
# 取得 Json 格式資料
json_data = requests.get(f'https://ess.api.cnyes.com/ess/api/'
                f'v1/news/keyword?q={stock_id}&limit=20&page=1').json()
# 依照格式擷取資料
items=json_data['data']['items']
for item in items:
    # 網址、標題和日期
    news_id = item["newsId"]
    title = item["title"]
    publish_at = item["publishAt"]
    # 使用 UTC 時間格式
    utc_time = datetime.utcfromtimestamp(publish_at)
    formatted_date = utc_time.strftime('%Y-%m-%d')
    # 前往網址擷取內容
    url = requests.get(f'https://news.cnyes.com/'
                       f'news/id/{news_id}').content
    soup = BeautifulSoup(url, 'html.parser')
    p_elements=soup .find_all('p')
    # 提取段落内容
    p=''
    for paragraph in p_elements[4:]:
        p+=paragraph.get_text()
    data.append([stock_id, formatted_date ,title,p])
# 建立表格
df = pd.DataFrame(data,columns=field)
df

,股號,日期,標題,內容
0,2330,2024-05-20,【量大強漲股整理】520後行情值得期待?機器人、CPO股誰最有潛力，將成為未來主流?,權值股方面，(2330-TW) 台積電，平盤作收，收在 835 元，在全球 AI 發展趨勢下...
1,2330,2024-05-20,台積電:本公司代子公司 TSMC Global Ltd. 公告取得固定收益證券,2.交易日期:113/5/7~113/5/203.交易數量、每單位價格及交易總金額:2515...
2,2330,2024-05-17,台股漲多拉回？軍工股520後還是資金避風港？,每日盤勢重點如何快速掌握？台股開盤日早上 8:00 準時鎖定「今天 Shot 這盤」延伸閱讀...
3,2330,2024-05-16,COMPUTEX展 聚焦AI PC供應鏈,首先來看看四大 CEO 將在 COMPUTEX 的主題演講內容。超微執行長蘇姿丰將探討 AM...
4,2330,2024-05-14,台積電:本公司代子公司 TSMC Global Ltd. 公告取得固定收益證券,2.交易日期:113/5/14~113/5/143.交易數量、每單位價格及交易總金額:060...
5,2330,2024-05-13,航運、橡膠與半導體族群買盤點火！台股剩12千金？今日盤後精選話題一文掌握,每日盤勢重點如何快速掌握？台股開盤日早上 8:00 準時鎖定「今天 Shot 這盤」延伸閱讀...
6,2330,2024-05-11,此前交易FTM獲利167萬美元的聰明錢地址近期已買入MKR、FTM和PEPE三種山寨幣,上一篇下一篇
7,2330,2024-05-11,此前交易FTM獲利167萬美元的聰明錢近期已買入MKR、FTM和PEPE三種山寨幣,上一篇下一篇
8,2330,2024-05-10,台積電:本公司受邀參加機構投資人說明會,1.召開法人說明會之日期：113/05/27 ~ 113/05/292.召開法人說明會之時間...
9,2330,2024-05-10,台積電:本公司代子公司 TSMC Global Ltd. 公告取得固定收益證券,2.交易日期:113/5/10~113/5/103.交易數量、每單位價格及交易總金額:XS2...


### 8️⃣  安裝及匯入套件

In [8]:
!pip install selenium
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')  # 不顯示瀏覽器
chrome_options.add_argument('--no-sandbox')# 以最高權限運行

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.7/467.7 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.1 MB/s eta 0:00:00


### 9️⃣ 使用 Selenium 取得股票新聞

In [9]:
# 透過 options 設定 driver
driver = webdriver.Chrome(options=chrome_options)
data2=[] # 表格數據
# 目標網址
url = f"https://www.cnyes.com/search/news?keyword={stock_id}"
driver.get(url)

# 模擬滑動滑鼠滾輪的行為，用於加載更多內容
scroll_pause_time = 2  # 等待時間
last_height = driver.execute_script(
                    "return document.body.scrollHeight")
while True:
    driver.execute_script(
        "window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)
    new_height = driver.execute_script(
        "return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

elements = driver.find_elements("xpath",
                        '//*[@id="_SearchAll"]/section/div/a')

# 擷取網址和標題
for element in elements:
    link = element.get_attribute("href")
    title = element.text
    title=title.split('\n')
    data2.append([stock_id, title[1] ,title[0],link])
# 關閉瀏覽器
driver.quit()
for link in data2:
  # 使用 requests 前往網址擷取新聞內容
  link_a = requests.get(link[3]).content
  link_b = BeautifulSoup(link_a,'html.parser')
  p_elements = link_b.find('article')
  # 取得段落内容
  try:
    link[3] = p_elements.get_text()
  except:
    link[3] = '無內容'
# 建立表格
df = pd.DataFrame(data2,columns=field)
df.tail()

KeyboardInterrupt: 

```
#把資料存成 csv 可以用以下這段
df.to_csv('/content/news_data.csv' )
```

#3.3 用 Python 套件輕鬆取得股市資料

##使用 yfinance 下載股市資料

### 🔟  安裝及匯入套件

In [10]:
!pip install yfinance
import yfinance as yf

### 1️⃣1️⃣  設定股票代碼和起止時間

In [11]:
# 指定要下載的股票代碼, 上市為 .TW;上櫃為 .TWO
stock_id = '2330.TW'
# 設定開始與結束時間
end = dt.date.today()
start = end - dt.timedelta(days=360)

### 1️⃣2️⃣ 取得每日股價 (開高低收) 資料

In [12]:
stock_data = yf.download(stock_id, start=start, end=end)
stock_data.tail()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-05-14,816.0,825.0,811.0,825.0,825.0,28691846
2024-05-15,838.0,844.0,837.0,839.0,839.0,39467772
2024-05-16,852.0,856.0,837.0,841.0,841.0,43927195
2024-05-17,848.0,848.0,834.0,835.0,835.0,26207289
2024-05-20,834.0,838.0,822.0,835.0,835.0,28808661


from matplotlib import pyplot as plt
_df_0['Open'].plot(kind='hist', bins=20, title='Open')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['High'].plot(kind='hist', bins=20, title='High')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['Low'].plot(kind='hist', bins=20, title='Low')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['Date']
  ys = series['Close']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_11.sort_values('Date', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Date')
_ = plt.ylabel('Close')

from matplotlib import pyplot as plt
_df_12['Open'].plot(kind='line', figsize=(8, 4), title='Open')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_13['High'].plot(kind='line', figsize=(8, 4), title='High')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_14['Low'].plot(kind='line', figsize=(8, 4), title='Low')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_15['Close'].plot(kind='line', figsize=(8, 4), title='Close')
plt.gca().spines[['top', 'right']].set_visible(False)



```
# 依照資料期間下載
stock_data = yf.download(stock_id, period="3mo")

# 下載不同時間頻率的資料 (1分K)
stock_data = yf.download(stock_id, interval="1m")
```



In [13]:
# 依照資料期間下載
stock_data = yf.download(stock_id, period="3mo")

# 下載不同時間頻率的資料 (1分K)
stock_data = yf.download(stock_id, interval="1m")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


### 1️⃣3️⃣ 取得多檔股票的資料

In [14]:
stocks = [stock_id, '2303.TW', '2454.TW'] #分別為台積電、聯電和聯發科
stock_data = yf.download(stocks, start=start, end=end)
stock_data.tail()

[*********************100%%**********************]  3 of 3 completed


Price       Adj Close                      Close                       High  \
Ticker        2303.TW 2330.TW 2454.TW    2303.TW 2330.TW 2454.TW    2303.TW   
Date                                                                          
2024-05-14  52.099998   825.0  1155.0  52.099998   825.0  1155.0  52.400002   
2024-05-15  51.799999   839.0  1155.0  51.799999   839.0  1155.0  52.400002   
2024-05-16  52.099998   841.0  1190.0  52.099998   841.0  1190.0  52.299999   
2024-05-17  52.799999   835.0  1165.0  52.799999   835.0  1165.0  52.799999   
2024-05-20  53.099998   835.0  1190.0  53.099998   835.0  1190.0  53.099998   

Price                             Low                       Open          \
Ticker     2330.TW 2454.TW    2303.TW 2330.TW 2454.TW    2303.TW 2330.TW   
Date                                                                       
2024-05-14   825.0  1160.0  51.799999   811.0  1110.0  52.299999   816.0   
2024-05-15   844.0  1175.0  51.799999   837.0  1145.0  52.299999   838.0   
2024-05-16   856.0  1210.0  51.900002   837.0  1175.0  52.099998   852.0   
2024-05-17   848.0  1185.0  51.799999   834.0  1165.0  52.299999   848.0   
2024-05-20   838.0  1195.0  52.500000   822.0  1165.0  53.000000   834.0   

Price                 Volume                      
Ticker     2454.TW   2303.TW   2330.TW   2454.TW  
Date                                              
2024-05-14  1115.0  44237062  28691846   7534580  
2024-05-15  1170.0  44199438  39467772   5522444  
2024-05-16  1180.0  41857613  43927195  10020318  
2024-05-17  1180.0  61246860  26207289   6360397  
2024-05-20  1165.0  55815584  28808661   6031212

### 1️⃣4️⃣ 取得公司基本資料


In [15]:
stock = yf.Ticker(stock_id)
stock.info # 為字典型態

{'address1': 'Hsinchu Science Park',
 'address2': 'No. 8, Li-Hsin Road 6',
 'city': 'Hsinchu City',
 'zip': '300096',
 'country': 'Taiwan',
 'phone': '886 3 563 6688',
 'fax': '886 3 563 7000',
 'website': 'https://www.tsmc.com',
 'industry': 'Semiconductors',
 'industryKey': 'semiconductors',
 'industryDisp': 'Semiconductors',
 'sector': 'Technology',
 'sectorKey': 'technology',
 'sectorDisp': 'Technology',
 'longBusinessSummary': 'Taiwan Semiconductor Manufacturing Company Limited, together with its subsidiaries, manufactures, packages, tests, and sells integrated circuits and other semiconductor devices in Taiwan, China, Europe, the Middle East, Africa, Japan, the United States, and internationally. It provides a range of wafer fabrication processes, including processes to manufacture complementary metal- oxide-semiconductor (CMOS) logic, mixed-signal, radio frequency, embedded memory, bipolar CMOS mixed-signal, and others. The company also offers customer and engineering support se

### 1️⃣5️⃣  取得損益表

In [16]:
financials = stock.financials
financials.tail()

,2023-12-31,2022-12-31,2021-12-31,2020-12-31
Other Gand A,60872800000.0,53524800000.0,36929600000.0,28457600000.0
Gross Profit,1175110600000.0,1348354800000.0,819537300000.0,711130100000.0
Cost Of Revenue,986625200000.0,915536500000.0,767877700000.0,628124700000.0
Total Revenue,2161735800000.0,2263891300000.0,1587415000000.0,1339254800000.0
Operating Revenue,2161735800000.0,2263891300000.0,1587415000000.0,1339254800000.0


In [17]:
bs = stock.balance_sheet
bs

,2023-12-31,2022-12-31,2021-12-31,2020-12-31
Treasury Shares Number,0.0,NaN,NaN,NaN
Ordinary Shares Number,25932070992.0,25930380458.0,25930380458.0,25930380458.0
Share Issued,25932070992.0,25930380458.0,25930380458.0,25930380458.0
Total Debt,956257900000.0,888174400000.0,753631900000.0,367792300000.0
Tangible Book Value,3406755700000.0,2877020500000.0,2122438100000.0,1809043200000.0
...,...,...,...,...
Cash Cash Equivalents And Short Term Investments,1714803200000.0,1586500100000.0,1204913700000.0,799893200000.0
Other Short Term Investments,249375400000.0,243686000000.0,139923500000.0,139722600000.0
Cash And Cash Equivalents,1465427800000.0,1342814100000.0,1064990200000.0,660170600000.0
Cash Equivalents,12326200000.0,13522700000.0,6182100000.0,6590000000.0


### 1️⃣6️⃣ 取得法人持股比例
因 yfinance 資料問題，此儲存格暫時無法使用

In [18]:
institutional_holders = stock.institutional_holders
institutional_holders.tail()

YFinanceDataException: Failed to parse holders json data.

## 使用 FinMind 下載股市資料

### 1️⃣7️⃣  安裝及匯入套件

In [19]:
!pip install FinMind
from FinMind.data import DataLoader
import getpass

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.3/146.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 16.4 MB/s eta 0:00:00
  Created wheel for ta: filename=ta-0.5.25-py3-none-any.whl size=24861 sha256=43af70b760c6b8f24b0ca6bb24aaa34b86c6ac9e542fd75c27750a8f9850937b
  Stored in directory: /root/.cache/pip/wheels/d4/bd/c4/0dda911b5461ee856352cd8c9e16472229bb51c95e74b74c08
Successfully built ta


### 1️⃣8️⃣  輸入 FinMind API 和帳號密碼

In [28]:
from google.colab import userdata
token = userdata.get('FinMind')
#token = getpass.getpass("請輸入 FinMind 金鑰：")

### 1️⃣9️⃣  建立 FinMind 資料庫物件和登入 FinMind

In [29]:
api = DataLoader()
api.login_by_token(api_token=token)

### 2️⃣0️⃣ 取得股價資料

In [30]:
# 股票代號
stock_id = '2330'
# 資料期間
end = dt.date.today()
start = end - dt.timedelta(days=360)

stock_data =  api.taiwan_stock_daily(
    stock_id=stock_id,
    start_date=start,
    end_date=end)

stock_data.tail()

2024-05-21 02:19:17.799 | INFO     | FinMind.data.finmind_api:get_data:125 - download TaiwanStockPrice, data_id: 2330


,date,stock_id,Trading_Volume,Trading_money,open,max,min,close,spread,Trading_turnover
233,2024-05-14,2330,29663617,24309306615,816.0,825.0,811.0,825.0,6.0,34685
234,2024-05-15,2330,41805778,35112739055,838.0,844.0,837.0,839.0,14.0,57892
235,2024-05-16,2330,46276890,39179826883,852.0,856.0,837.0,841.0,2.0,60617
236,2024-05-17,2330,27855719,23373478247,848.0,848.0,834.0,835.0,6.0,38718
237,2024-05-20,2330,30148330,25069342221,834.0,838.0,822.0,835.0,0.0,43094


### 2️⃣1️⃣ 取得損益表資料

In [31]:
financial_data = api.taiwan_stock_financial_statement(
    stock_id=stock_id,
    start_date=str(start),)

financial_data.tail()

2024-05-21 02:19:24.050 | INFO     | FinMind.data.finmind_api:get_data:125 - download TaiwanStockFinancialStatements, data_id: 2330


,date,stock_id,type,value,origin_name
59,2024-03-31,2330,TotalNonoperatingIncomeAndExpense,1.752490e+10,營業外收入及支出
60,2024-03-31,2330,OtherComprehensiveIncome,4.127395e+10,其他綜合損益（淨額）
61,2024-03-31,2330,EquityAttributableToOwnersOfParent,2.673986e+11,綜合損益總額歸屬於母公司業主
62,2024-03-31,2330,NoncontrollingInterests,-9.033610e+08,綜合損益總額歸屬於非控制權益
63,2024-03-31,2330,GrossProfit,3.145053e+11,營業毛利（毛損）


### 2️⃣2️⃣ 取得法人買賣資料

In [32]:
investors_data = api.taiwan_stock_institutional_investors(
    stock_id=stock_id,
    start_date=str(start),)
investors_data.tail()

2024-05-21 02:19:28.314 | INFO     | FinMind.data.finmind_api:get_data:125 - download TaiwanStockInstitutionalInvestorsBuySell, data_id: 2330


,date,stock_id,buy,name,sell
1185,2024-05-20,2330,0,Foreign_Dealer_Self,0
1186,2024-05-20,2330,1482532,Dealer_self,294536
1187,2024-05-20,2330,724049,Dealer_Hedging,193620
1188,2024-05-20,2330,14991881,Foreign_Investor,18083339
1189,2024-05-20,2330,243000,Investment_Trust,173369


## 使用 FinLab 下載股市資料

### 2️⃣3️⃣ 安裝及匯入套件

In [34]:
!pip install finlab
import finlab
from finlab import data

Mounted at /content/drive


### 2️⃣4️⃣ 登入 FinLab

In [35]:
from google.colab import userdata
token = userdata.get('FinLab')
finlab.login(token)

輸入成功!


### 2️⃣5️⃣ 取得收盤價

In [36]:
close = data.get('price:收盤價')
close.tail()

Daily usage: 26.0 / 5000 MB - price:收盤價


symbol,0015,0050,0051,0052,0053,0054,0055,0056,0057,0058,...,9944,9945,9946,9949,9950,9951,9955,9958,9960,9962
date,,,,,,,,,,,,,,,,,,,,,
2024-05-14,NaN,164.40,79.20,159.55,88.20,NaN,26.29,39.26,121.25,NaN,...,22.25,42.65,37.25,23.1,15.80,70.6,29.55,248.5,27.0,16.50
2024-05-15,NaN,166.65,79.40,161.65,89.75,NaN,26.62,39.30,123.05,NaN,...,22.90,46.55,36.85,23.1,15.70,70.5,29.90,240.5,27.0,16.65
2024-05-16,NaN,167.45,80.10,161.90,90.30,NaN,26.80,39.60,123.60,NaN,...,23.35,46.90,38.20,23.2,15.55,70.6,30.85,243.0,27.0,16.65
2024-05-17,NaN,167.25,80.00,161.70,89.75,NaN,26.95,39.51,123.30,NaN,...,23.25,46.60,39.75,23.1,15.65,70.7,30.90,252.5,27.0,16.70
2024-05-20,NaN,167.20,80.05,161.20,89.30,NaN,27.06,39.66,122.50,NaN,...,23.40,45.30,39.90,23.2,15.75,70.9,33.95,250.5,27.0,17.05


### 2️⃣6️⃣ 選擇產業

In [37]:
data.set_universe(market='TSE', category='半導體')
close = data.get('price:收盤價')
close.tail()

Daily usage: 26.2 / 5000 MB - security_categories


symbol,2302,2303,2329,2330,2337,2338,2340,2342,2344,2351,...,6937,8016,8028,8081,8110,8131,8150,8162,8261,8271
date,,,,,,,,,,,,,,,,,,,,,
2024-05-14,17.40,52.1,58.9,825.0,27.4,68.2,37.55,32.15,25.60,109.5,...,197.5,264.5,52.9,288.5,17.55,38.05,43.45,38.50,83.3,64.8
2024-05-15,17.60,51.8,59.1,839.0,27.1,68.5,37.85,31.75,25.50,115.0,...,196.0,266.5,52.5,292.0,18.85,38.30,43.45,38.35,82.7,64.4
2024-05-16,17.55,52.1,59.6,841.0,27.2,69.3,38.20,31.50,25.60,116.5,...,194.0,268.0,53.5,298.0,18.95,38.50,43.65,39.50,83.3,64.2
2024-05-17,18.35,52.8,60.5,835.0,27.3,70.0,38.50,31.50,25.35,118.5,...,195.5,262.5,58.8,297.5,18.50,38.50,43.20,38.80,82.6,65.3
2024-05-20,18.00,53.1,60.3,835.0,27.7,71.3,38.30,31.70,25.50,115.5,...,193.0,256.5,58.4,295.0,18.45,38.65,43.85,39.40,82.3,65.4


### 2️⃣7️⃣ 取得財報資料

In [38]:
df = data.get('financial_statement:每股盈餘')
df.tail()

Daily usage: 28.9 / 5000 MB - financial_statement:每股盈餘


symbol,2302,2303,2329,2330,2337,2338,2340,2342,2344,2351,...,6937,8016,8028,8081,8110,8131,8150,8162,8261,8271
date,,,,,,,,,,,,,,,,,,,,,
2023-Q1,0.06,1.31,0.25,7.98,-0.19,1.01,0.16,-0.30,-0.25,1.21,...,NaN,2.48,0.40,2.97,0.07,0.52,0.28,NaN,0.49,1.05
2023-Q2,0.25,1.27,0.72,7.01,0.04,0.49,0.26,0.02,0.09,1.11,...,1.15,4.93,0.87,4.90,-0.08,0.52,0.86,-0.90,0.59,0.96
2023-Q3,0.16,1.29,0.82,8.14,-0.22,-0.40,0.31,-0.05,-0.03,1.14,...,1.47,4.51,0.76,6.09,0.03,0.10,0.80,0.05,1.16,1.14
2023-Q4,0.04,1.06,0.86,9.21,-0.55,0.68,-0.03,-0.79,-0.10,0.61,...,2.40,3.58,-0.01,3.37,-0.04,0.06,0.66,0.46,0.58,1.36
2024-Q1,0.11,0.84,0.70,8.70,-0.58,3.16,-0.08,-0.06,-0.11,0.90,...,1.25,3.62,0.36,4.41,0.30,0.79,0.60,0.17,0.74,0.94


### 2️⃣8️⃣  取得法人資料

In [39]:
df = data.get('institutional_investors_trading_summary:投信買賣超股數')
df.tail()

Daily usage: 37.0 / 5000 MB - institutional_investors_trading_summary:投信買賣超股數


symbol,2302,2303,2329,2330,2337,2338,2340,2342,2344,2351,...,6937,8016,8028,8081,8110,8131,8150,8162,8261,8271
date,,,,,,,,,,,,,,,,,,,,,
2024-05-14,0.0,14538337.0,-6000.0,-14546.0,429000.0,0.0,0.0,0.0,-4517.0,381000.0,...,0.0,3430.0,0.0,18000.0,0.0,0.0,56454.0,0.0,0.0,0.0
2024-05-15,0.0,-181510.0,-5000.0,-15744.0,198000.0,0.0,0.0,0.0,-22775.0,118000.0,...,0.0,10923.0,0.0,17000.0,0.0,3220.0,97895.0,0.0,0.0,34000.0
2024-05-16,0.0,-1208789.0,-4000.0,-292496.0,268000.0,0.0,0.0,0.0,-184345.0,44000.0,...,0.0,19991.0,0.0,34000.0,0.0,0.0,-85953.0,0.0,0.0,99000.0
2024-05-17,0.0,20658104.0,-3000.0,117392.0,1471910.0,0.0,0.0,0.0,-240503.0,416000.0,...,0.0,-743647.0,0.0,193662.0,0.0,0.0,-4892574.0,0.0,0.0,182636.0
2024-05-20,0.0,22450193.0,-4000.0,69631.0,1948000.0,0.0,0.0,0.0,-8172.0,195000.0,...,0.0,-998689.0,0.0,88000.0,0.0,2722.0,-5121000.0,0.0,0.0,0.0


#3.4 SQL資料庫

### 2️⃣9️⃣ 匯入套件及連線資料庫

In [40]:
import sqlite3
conn = sqlite3.connect('stock.db')

### 3️⃣0️⃣設定資料庫結構

In [41]:
cursor = conn.cursor()
cursor.execute('''
CREATE TABLE IF NOT EXISTS 日頻資料 (
    sno INTEGER PRIMARY KEY AUTOINCREMENT,
    Stock_Id TEXT,
    Date DATE,
    Open FLOAT,
    High FLOAT,
    Low FLOAT,
    Close FLOAT,
    Adj_Close FLOAT,
    Volume INTEGER
);
''')
conn.commit()

### 3️⃣1️⃣ 傳入資料到資料庫

In [42]:
df = yf.download('2330.TW',start='2023-08-01')
df = df.reset_index()
df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
df.rename(columns={"Adj Close": "Adj_Close"}, inplace=True)
df.insert(0,'Stock_id','2330')

df.to_sql('日頻資料',conn,if_exists='append',index=False)

[*********************100%%**********************]  1 of 1 completed


195

### 3️⃣2️⃣ 查詢表格資料

In [43]:
def table_info(table_name):
    cursor = conn.cursor()
    cursor.execute(f"PRAGMA table_info({table_name})")
    columns = cursor.fetchall()
    column_names = [column[1] for column in columns]
    print(f"資料庫表 '{table_name}' 的欄位名稱：", column_names)
    all_data = conn.execute(f'SELECT * FROM {table_name}')
    for i in all_data.fetchall():
        print(i)

# 查詢表格資料
table_info("日頻資料")

資料庫表 '日頻資料' 的欄位名稱： ['sno', 'Stock_Id', 'Date', 'Open', 'High', 'Low', 'Close', 'Adj_Close', 'Volume']
(1, '2330', '2023-08-01', 565.0, 568.0, 564.0, 567.0, 558.317138671875, 16259643)
(2, '2330', '2023-08-02', 567.0, 569.0, 558.0, 561.0, 552.4089965820312, 25583234)
(3, '2330', '2023-08-04', 556.0, 560.0, 552.0, 554.0, 545.5161743164062, 26279173)
(4, '2330', '2023-08-07', 558.0, 561.0, 556.0, 558.0, 549.4548950195312, 14369551)
(5, '2330', '2023-08-08', 558.0, 558.0, 551.0, 552.0, 543.546875, 19010690)
(6, '2330', '2023-08-09', 550.0, 557.0, 550.0, 554.0, 545.5161743164062, 14389090)
(7, '2330', '2023-08-10', 552.0, 554.0, 550.0, 551.0, 542.5621337890625, 17892615)
(8, '2330', '2023-08-11', 556.0, 558.0, 546.0, 546.0, 537.6387329101562, 17608427)
(9, '2330', '2023-08-14', 540.0, 543.0, 537.0, 541.0, 532.71533203125, 20939030)
(10, '2330', '2023-08-15', 543.0, 545.0, 540.0, 542.0, 533.6998901367188, 14066307)
(11, '2330', '2023-08-16', 541.0, 543.0, 536.0, 542.0, 533.6998901367188, 282

### 3️⃣3️⃣ 新增資料

In [44]:
def insert_data(stock_id, start):
  # 下載資料
  df = yf.download(f'{stock_id}.TW',start=start)
  df = df.reset_index()
  df.rename(columns={"Adj Close": "Adj_Close"}, inplace=True)
  df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
  df.insert(0,'Stock_Id',stock_id)

  # 新增資料表
  df.to_sql('日頻資料',conn,if_exists='append',index=False)

# 新增 2317 資料
insert_data(stock_id=2317, start='2023-08-01')

[*********************100%%**********************]  1 of 1 completed


### 3️⃣4️⃣ 從資料庫取得資料

In [45]:
query = ("SELECT Stock_id, Date, Close "
         "FROM 日頻資料 "
         "WHERE Date < '2023-08-15' AND Stock_id = '2317'")
df = pd.read_sql(query, conn, parse_dates=['Date'])
df.tail()

,Stock_Id,Date,Close
4,2317,2023-08-08,110.5
5,2317,2023-08-09,110.5
6,2317,2023-08-10,110.0
7,2317,2023-08-11,108.5
8,2317,2023-08-14,110.0


### 3️⃣5️⃣ 修改並關閉資料庫

In [46]:
conn.commit()
conn.close()